In [4]:
import psycopg2
import requests
import json
import pandas as pd
from datetime import datetime

#Credenciales Redshift
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
port = 5439
user = 'franciscorossi48_coderhouse'
password = 'jhu3Yv805G'
database = 'data-engineer-database'

#Clave Alpha Vantage
api_key = 'QSZ491MBGEV1MZG8'
symbol = 'MELI'

#Rango de fechas
start_date = '2022-12-01'
end_date = '2023-01-01'

#Endpoint para cotizaciones diarias
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&outputsize=full'

#Solicitud a la API
response = requests.get(url)
data_json = response.json()

#Extracción
time_series_data = data_json.get('Time Series (Daily)', [])
data_list = []

for timestamp, values in time_series_data.items():
    date_time_obj = datetime.strptime(timestamp, '%Y-%m-%d')
    
    #Filtrar por fechas
    if start_date <= timestamp <= end_date:
        data_list.append([date_time_obj, float(values['1. open']), float(values['2. high']),
                          float(values['3. low']), float(values['4. close']), int(values['5. volume'])])

#DF
column_names = ['fecha', 'apertura', 'alto', 'bajo', 'cierre', 'volumen']
df = pd.DataFrame(data_list, columns=column_names)

#Eliminar duplicados
df.drop_duplicates(subset=['fecha'], inplace=True)

print(df)

        fecha  apertura      alto      bajo  cierre  volumen
0  2022-12-30    846.81  851.0578  830.0100  846.24   365606
1  2022-12-29    849.47  874.0750  838.0450  866.28   444056
2  2022-12-28    856.44  867.9900  838.2500  839.57   475520
3  2022-12-27    872.99  874.3800  848.1250  854.30   294073
4  2022-12-23    864.30  879.7300  853.0800  878.32   288078
5  2022-12-22    882.97  885.3900  863.4700  873.26   470462
6  2022-12-21    877.58  908.9900  873.0000  900.10   409859
7  2022-12-20    881.18  907.0799  874.8700  875.91   540260
8  2022-12-19    844.24  895.8000  843.5200  884.27   723840
9  2022-12-16    830.00  849.6100  825.0000  843.98   731019
10 2022-12-15    852.07  854.9000  818.2200  835.09   652056
11 2022-12-14    864.06  887.9900  855.0000  867.30   579648
12 2022-12-13    931.48  942.0900  867.3000  870.65   880526
13 2022-12-12    885.28  894.3700  868.4950  880.25   518156
14 2022-12-09    878.37  908.2466  872.0600  896.48   409557
15 2022-12-08    872.00 

In [5]:
df.head()

,fecha,apertura,alto,bajo,cierre,volumen
0,2022-12-30,846.81,851.0578,830.010,846.24,365606
1,2022-12-29,849.47,874.0750,838.045,866.28,444056
2,2022-12-28,856.44,867.9900,838.250,839.57,475520
3,2022-12-27,872.99,874.3800,848.125,854.30,294073
4,2022-12-23,864.30,879.7300,853.080,878.32,288078


In [6]:
from sqlalchemy import create_engine

#Credenciales Redshift
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
port = 5439
user = 'franciscorossi48_coderhouse'
password = 'jhu3Yv805G'
database = 'data-engineer-database'

#Conexión a Redshift
engine = create_engine(f'redshift+psycopg2://{user}:{password}@{host}:{port}/{database}')

table_name = 'ejemplo_finanzas'

#Cargar el DF en Redshift
df.to_sql(table_name, engine, schema='franciscorossi48_coderhouse', if_exists='replace', index=False)

print(f"Datos cargados exitosamente en la tabla {table_name} en Redshift.")


Datos cargados exitosamente en la tabla ejemplo_finanzas en Redshift.
